In [1]:
import numpy as np
from matplotlib import pyplot as plt
#from matplotlib import rcParams
from decimal import Decimal # for rouinding the d range

import plotly
import plotly.graph_objects as go

from sim_handling import Simulation,loadSim
from diffusionstuff7 import f0d,f1d,f2d

C:\Users\Max-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\llvmlite\llvmpy\__init__.py:3: UserWarning: The module `llvmlite.llvmpy` is deprecated and will be removed in the future.
  warnings.warn(
C:\Users\Max-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\llvmlite\llvmpy\core.py:8: UserWarning: The module `llvmlite.llvmpy.core` is deprecated and will be removed in the future. Equivalent functionality is provided by `llvmlite.ir`.
  warnings.warn(
C:\Users\Max-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\llvmlite\llvmpy\passes.py:17: UserWarning: The module `llvmlite.llvmpy.passes` is deprecated and will be removed in the future. If you are using this code, it should be inlined into your own project.
  warnings.warn(


In [2]:
%matplotlib widget

This notebook is to create 3d plots of the number of steps (z axis) over time (y axis) for 1D simulations against a variable parameter (x axis). The parameters to begin with will be those from the 1D QLL mediated continuum model of ice. 
A 3d plot will be created for each parameter. Additionally, the fig11 x value and predicted Nss value will be plotted on a slice of the 3d plot at the final x value, and a difference between the actual and predicted values will printed.
Around 20 values for each parameter will be tested.

In [3]:
#parameters for all simulations
number_of_timesteps = 110_000 #its just steady state past 100k timesteps in all tested cases
#we want all sims to be the same length, so we are going to use the same number of timesteps for all sims (not using # of layers halt or discretization halt)
use_layers = False
discretization_halt = False 

print_ranges = False
plot_Nss_vs_params = False

#number of values to test for each parameter
num_sims = 23

sigma_range = np.linspace(0.19, 0.5, num_sims)#- 0.19 INSTEAD OF 0.2, OTHERWISE same as in Nesh et al 2016

c_r_range = np.linspace(0.15, 0.7, num_sims)#- same as in Nesh et al 2016

d_range = np.linspace(0.75, 6, num_sims)#-*E-4,  same as in Nesh et al 2016
d_range = [round(d,2)*1E-4 for d in d_range] # round to nearest .01
d_range = [float(Decimal(str(x)).quantize(Decimal('1.00E-4'))) for x in d_range]

nu_kin_range = np.linspace(20, 140, num_sims)#- same as in Nesh et al 2016
nu_kin_range = [int(round(nukin)) for nukin in nu_kin_range] # round to nearest 1

l_range = np.linspace(350, 850, num_sims) # 0.2 microns per point means L of 17.5 microns to 42.5 microns - same as in Nesh et al 2016
l_range = [int(round(l,-1)) for l in l_range] # round to nearest 10


if print_ranges:
    print("sigma range: ", sigma_range)
    print("c_r range: ", c_r_range)
    print("d range: ", d_range)
    print("nu_kin range: ", nu_kin_range)
    print("l range: ", l_range)



In [ ]:
#sigma sims
Ntots = []

for sigma in sigma_range:
#set up and run and plot a new simulation
	sigma_sim_1d = Simulation(f1d, (500,), method='RK45')

	#set it based off number of timesteps so that it's the same for all (different growth rates)
	sigma_sim_1d.uselayers = use_layers
	sigma_sim_1d.countermax = number_of_timesteps #enough for steady state under original regular conditions
	sigma_sim_1d.discretization_halt = discretization_halt # needs to be same length of simulation, we want to see the differences if it causes discretization halt

	#test_1d_asym.float_params['DoverdeltaX2'] 
	#sigma_sim_1d.layermax = 1100 
	sigma_sim_1d.sigmastepmax = sigma

	Ntots.append(sigma_sim_1d.getNtot())

In [ ]:
#process the results
zacc = []
#turn Ntot into Nss
for Ntot in Ntots:
    maxes = np.array([np.max(n) for n in Ntot[:]])
    mins = np.array([np.min(n) for n in Ntot[:]])
    z = maxes-mins #(array of size x (number of timesteps), each value is z value (number of steps on surface) at that time)
    zacc.append(z)

zacc = np.array(zacc)
print(zacc)

#stack z frames together along x axis, so time is y axis and different sigma values are x axis
zs = np.stack(zacc, axis=0)
print(zacc.shape)

ys = np.linspace(0, zacc.shape[1], zacc.shape[1])

ys,xs = np.meshgrid(ys,sigma_range)

In [ ]:
#matplotlib 3d plot of the results - slow
# fig = plt.figure('number of steps over time vs sigma')
# ax = plt.axes(projection='3d')
# ax.set_xlabel('Sigma')
# ax.set_ylabel('Time')
# ax.set_zlabel('Number of steps (bilyers of ice)')

# print(len(Ntots))

# ax.plot_wireframe(X=xs, Y=ys, Z=zs, cmap='viridis')#, vmin=0, vmax=200)

In [ ]:
#3d plot of results. plotly
if plot_Nss_vs_params:
    fig = go.Figure(data=[go.Surface(x=xs,y=ys,z=zs, colorscale="blues", showscale=False)])
    fig.update_layout(scene = dict(
                        xaxis_title='Sigma corner',
                        yaxis_title='Time (timesteps)',
                        zaxis_title='Number of steps (bilyers of ice)'),
                        title='number of steps over time vs sigma')

In [ ]:
# get the # of steps at the last timestep of the simulation for each sigma value (for plotting and creating the NSS vs parameters plot (Fig 7 in Neshyba et al 2016))
final_Nss_vs_sigma = zs[:, -1]
np.save('final_Nss_vs_sigma', final_Nss_vs_sigma)

# plot the results using matplotlib
plt.close('number of steps after 110k steps vs sigma')
fig2 = plt.figure('number of steps after 110k steps vs sigma')
ax2 = plt.axes()
ax2.set_xlabel('Sigma')
ax2.set_ylabel('Number of steps (bilayers of ice)')
ax2.scatter(sigma_range, final_Nss_vs_sigma, s=50, marker='o', facecolors='none', edgecolors='b')
plt.grid()
plt.show()

# equation that relates the parameters to the number of steps (Nss) in the ice sheet - from Neshyba et al 2016, Figure 11
#NSS ≈ M *(v_kin*L^2*c_r^β*σ_I,corner)/ D + B

In [ ]:
#Save the plot as an interactive webpage html file
#plotly.offline.plot(fig, filename='Nss_vs_sigma.html')

In [5]:
#Do the same thing now, but for center reduction

#process the results
zacc = []

#run and process results

for center_reduction in c_r_range:
#set up and run and plot a new simulation
	test_1d_asym = Simulation(f1d, (500,), method='RK45')

	#set it based off number of timesteps so that it's the same for all (different growth rates)
	test_1d_asym.uselayers = use_layers
	test_1d_asym.countermax = number_of_timesteps #enough for steady state under original regular conditions

	test_1d_asym.discretization_halt = discretization_halt # needs to be same length of simulation, we want to see the differences if it causes discretization halt

	test_1d_asym.center_reduction = center_reduction

	Ntot= test_1d_asym.getNtot()
	maxes = np.array([np.max(n) for n in Ntot[:]])
	mins = np.array([np.min(n) for n in Ntot[:]])
	z = maxes-mins #(array of size x (number of timesteps), each value is z value (number of steps on surface) at that time)
	zacc.append(z)

zacc = np.array(zacc)
print(zacc)

#stack z frames together along x axis, so time is y axis and different sigma values are x axis
zs = np.stack(zacc, axis=0)
print(zacc.shape)

ys = np.linspace(0, zacc.shape[1], zacc.shape[1])

ys,xs = np.meshgrid(ys,c_r_range)

breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations


In [6]:
#save NSS vs c_r data for last step
final_Nss_vs_c_r = zs[:, -1]
np.save('final_Nss_vs_c_r', final_Nss_vs_c_r)

In [7]:
#3d plot of results. plotly
if plot_Nss_vs_params:
    fig = go.Figure(data=[go.Surface(x=xs,y=ys,z=zs, colorscale="blues", showscale=False)])
    fig.update_layout(scene = dict(
                        xaxis_title='Center reduction, %',
                        yaxis_title='Time (timesteps)',
                        zaxis_title='Number of steps (bilyers of ice)'),
                        title='number of steps over time vs center reduction')
    fig.show(width=1000, height=1000)
    #save the results as an interactive html file
    fig.write_html("c_r_vs_sigma.html")
    #plotly.offline.plot(fig, filename='c_r_vs_sigma.html')

In [8]:
#Do the same thing now, but for Diffusion coefficient


#process the results
zacc = []

#run and process results

for D in d_range:
#set up and run and plot a new simulation
	sim_1d_D = Simulation(f1d, (500,), method='RK45')

	#set it based off number of timesteps so that it's the same for all (different growth rates)
	sim_1d_D.uselayers = use_layers
	sim_1d_D.countermax = number_of_timesteps #enough for steady state under original regular conditions

	sim_1d_D.discretization_halt = discretization_halt # needs to be same length of simulation, we want to see the differences if it causes discretization halt


	deltaX = sim_1d_D.x[1]-sim_1d_D.x[0]
	sim_1d_D.float_params['DoverdeltaX2'] = D/deltaX**2

	sim_1d_D.layermax = 1100
	sim_1d_D.sigmastepmax = 0.20
	sim_1d_D.center_reduction = 0.25

	Ntot= sim_1d_D.getNtot()
	maxes = np.array([np.max(n) for n in Ntot[:]])
	mins = np.array([np.min(n) for n in Ntot[:]])
	z = maxes-mins #(array of size x (number of timesteps), each value is z value (number of steps on surface) at that time)
	zacc.append(z)

zacc = np.array(zacc)
print(zacc)

#stack z frames together along x axis, so time is y axis and different sigma values are x axis
zs = np.stack(zacc, axis=0)
print(zacc.shape)

ys = np.linspace(0, zacc.shape[1], zacc.shape[1])

ys,xs = np.meshgrid(ys,d_range)

breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations


In [9]:
#save NSS vs D data for last step
final_Nss_vs_D = zs[:, -1]
np.save('final_Nss_vs_D', final_Nss_vs_D)

In [ ]:
#3d plot of results. plotly
if plot_Nss_vs_params:
    fig = go.Figure(data=[go.Surface(x=xs,y=ys,z=zs, colorscale="blues", showscale=False)])
    fig.update_layout(scene = dict(
                        xaxis_title='D, microns^2/microsecond',
                        yaxis_title='Time (timesteps)',
                        zaxis_title='Number of steps (bilayers of ice)'),
                        title='number of steps over time vs diffusion coefficient')
    fig.show(width=1000, height=1000)
    #save the results as an interactive html file
    fig.write_html("D_vs_sigma.html")   

In [10]:
#Do the same thing now, but for nu_kin

#run and process results
zacc = []

for nu_kin in nu_kin_range:
#set up and run and plot a new simulation
	sim_1d_D = Simulation(f1d, (500,), method='RK45')

	#set it based off number of timesteps so that it's the same for all (different growth rates)
	sim_1d_D.uselayers = use_layers
	sim_1d_D.countermax = number_of_timesteps #enough for steady state under original regular conditions

	sim_1d_D.discretization_halt = discretization_halt # needs to be same length of simulation, we want to see the differences if it causes discretization halt

	#deprate is accesible, changable parameter, not in microns per second. That value of nu_kin in microns per second is set as 49, so we divide by 49 to get the correct range of deprate values
	sim_1d_D.float_params['deprate'] *= nu_kin/49

	#defaults
	sim_1d_D.layermax = 1100
	sim_1d_D.sigmastepmax = 0.20
	sim_1d_D.center_reduction = 0.25

	Ntot= sim_1d_D.getNtot()
	maxes = np.array([np.max(n) for n in Ntot[:]])
	mins = np.array([np.min(n) for n in Ntot[:]])
	z = maxes-mins #(array of size x (number of timesteps), each value is z value (number of steps on surface) at that time)
	zacc.append(z)

zacc = np.array(zacc)
print(zacc)

#stack z frames together along x axis, so time is y axis and different sigma values are x axis
zs = np.stack(zacc, axis=0)
print(zacc.shape)

ys = np.linspace(0, zacc.shape[1], zacc.shape[1])
ys,xs = np.meshgrid(ys,nu_kin_range)

breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations


In [11]:
# get the # of steps at the last timestep of the simulation for each sigma value (for plotting and creating the NSS vs parameters plot (Fig 7 in Neshyba et al 2016))
final_Nss_vs_v_kin = zs[:, -1]
np.save('final_Nss_vs_v_kin', final_Nss_vs_v_kin)

In [ ]:
#3d plot of results. plotly
if plot_Nss_vs_params:
    fig = go.Figure(data=[go.Surface(x=xs,y=ys,z=zs, colorscale="blues", showscale=False)])
    fig.update_layout(scene = dict(
                        xaxis_title='nu_kin, microns/second',
                        yaxis_title='Time (timesteps)',
                        zaxis_title='Number of steps (bilayers of ice)'),
                        title='number of steps over time vs nu_kin')
    #save the results as an interactive html file
    fig.write_html("nu_kin_vs_sigma.html")

In [12]:
#Do the same thing now, but for facet size



#run and process results
zacc = []
for l in l_range:
#set up and run and plot a new simulation
	sim_1d_D = Simulation(f1d, (l,), method='RK45')

	#set it based off number of timesteps so that it's the same for all (different growth rates)
	sim_1d_D.uselayers = use_layers
	sim_1d_D.countermax = number_of_timesteps #enough for steady state under original regular conditions

	sim_1d_D.discretization_halt = discretization_halt # needs to be same length of simulation, we want to see the differences if it causes discretization halt

	#deltaX = sim_1d_D.x[1]-sim_1d_D.x[0]
	#sim_1d_D.float_params['DoverdeltaX2'] = D/deltaX**2

	sim_1d_D.layermax = 1100
	sim_1d_D.sigmastepmax = 0.20
	sim_1d_D.center_reduction = 0.25

	Ntot = sim_1d_D.getNtot()
	maxes = np.array([np.max(n) for n in Ntot[:]])
	mins = np.array([np.min(n) for n in Ntot[:]])
	z = maxes-mins #(array of size x (number of timesteps), each value is z value (number of steps on surface) at that time)
	zacc.append(z)

zacc = np.array(zacc)
print(zacc)

#stack z frames together along x axis, so time is y axis and different sigma values are x axis
zs = np.stack(zacc, axis=0)
print(zacc.shape)

ys = np.linspace(0, zacc.shape[1], zacc.shape[1])

ys,xs = np.meshgrid(ys,l_range)

breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations
breaking because reached max number of iterations


In [13]:
# get the # of steps at the last timestep of the simulation for each sigma value (for plotting and creating the NSS vs parameters plot (Fig 7 in Neshyba et al 2016))
final_Nss_vs_L = zs[:, -1]
np.save('final_Nss_vs_L', final_Nss_vs_L)


In [ ]:
#3d plot of results. plotly
if plot_Nss_vs_params:
    fig = go.Figure(data=[go.Surface(x=xs,y=ys,z=zs, colorscale="blues", showscale=False)])
    fig.update_layout(scene = dict(
                        xaxis_title='L, num points, (10 points per micron)',
                        yaxis_title='Time (timesteps)',
                        zaxis_title='Number of steps (bilayers of ice)'),
                        title='number of steps over time vs facet size')

In [ ]:
#save the results as an interactive html file
fig.write_html("L_vs_sigma.html")

In [ ]:
 #set up and run and plot a new simulation
test_1d_asym = Simulation(f1d, (500,), method='RK45', rtol=1e-12)
#test_1d_asym = Simulation(f1d, (500,), method='LSODA')

test_1d_asym.discretization_halt = False

#try a noisy simulation
test_1d_asym.noisy_init = False

#test_1d_asym.float_params['DoverdeltaX2'] *= 2 # raised to lower parameter product value
test_1d_asym.layermax = 1100
#test_2d_asym.sigmastepmax= -0.01 #ablation
test_1d_asym.sigmastepmax = 0.195 #growth - lowered to lower parameter product value
test_1d_asym.sigmastepmax = 0.20 
test_1d_asym.center_reduction = 0.25
#test_2d_asym.center_reduction = 0.003 # lowered to lower parameter product value


In [ ]:
#plot the simulation at the end
test_1d_asym.plot(completion=1, ice=True, tot=True, figurename='ending surface')

In [ ]:
#graph number of steps over time
Ntot = test_1d_asym.getNtot()
#Ntot = cycle_sim.getNtot()
print(Ntot.shape)

maxes = np.array([np.max(n) for n in Ntot[:]])
mins = np.array([np.min(n) for n in Ntot[:]])

#print(maxes,mins)
plt.figure('Number of steps over time')
ax = plt.subplot(111)

ax.set_ylabel('Number of steps on surface')

if test_1d_asym.sigmastepmax < 0: #ablation case
    ax.set_xlabel('Total number of layers ablated')
    ax.plot( Ntot[::-1,0]+test_1d_asym.layermax,maxes-mins) #flipping to show ablation progress from left to right, adding layermax to show # layers ablated instead of negative # layers grown
else: #growth case   
    ax.set_xlabel('Total number of layers grown')
    ax.plot( Ntot[:,0],maxes-mins)


plt.grid()

In [ ]:
plt.close('Number of steps over time')

In [14]:
# Load the saved NSS values for each parameter
final_Nss_vs_sigma = np.load('final_Nss_vs_sigma.npy')
final_Nss_vs_v_kin = np.load('final_Nss_vs_v_kin.npy')
final_Nss_vs_L = np.load('final_Nss_vs_L.npy')
final_Nss_vs_c_r = np.load('final_Nss_vs_c_r.npy')
final_Nss_vs_D = np.load('final_Nss_vs_D.npy')

beta = 0.65
# Define the common X-axis for all variables
x = (nu_kin_range * l_range ** 2 * c_r_range ** beta * sigma_range[:, np.newaxis]) / d_range

# plot the results using matplotlib
plt.close('NSS vs parameters')
fig = plt.figure('NSS vs parameters')
ax = plt.axes()
ax.set_xlabel('(v_kin * L^2 * c_r^β * σ_I,corner) / D')
ax.set_ylabel('Number of steps (bilayers of ice)')
ax.scatter(x, final_Nss_vs_sigma, s=50, marker='o', facecolors='none', edgecolors='b', label='Sigma')
ax.scatter(x, final_Nss_vs_v_kin, s=50, marker='^', facecolors='none', edgecolors='r', label='v_kin')
ax.scatter(x, final_Nss_vs_L, s=50, marker='s', facecolors='none', edgecolors='g', label='L')
ax.scatter(x, final_Nss_vs_c_r, s=50, marker='D', facecolors='none', edgecolors='m', label='c_r')
ax.scatter(x, final_Nss_vs_D, s=50, marker='P', facecolors='none', edgecolors='k', label='D')
plt.legend()
plt.grid()
plt.show()


TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'

In [ ]:
#calculating Nss vs parameter thingy

# N_ss = # number of steps to steady state 
from pint import UnitRegistry; AssignQuantity = UnitRegistry().Quantity


#slope and intercept of linear fit of figure11 x to N_ss
#for sinusoidal growth
M = 0.0027 # bilayers / micron ????
M = 2.7 # what it actually looks like on the plot in the paper
B= 2.9 #bilayers
#for parabolic growth
#M = 0.0025
#B = 1.6

def Nss_fig11(M,B,x):
    return M*x + B

def figure11(L,D,c_r,nu_kin,sigma):
    """
    L: length of domain, microns
    D: diffusion coefficient, microns^2 / microsecond
    c_r: center reduction, fraction
    nu_kin: growth rate, microns / second
    sigma: supersaturation at corner, fraction
    """
    beta = 0.65
    return nu_kin*L**2*sigma*(c_r)**beta / D

#Lx,Ly = test_1d_asym.shape # num points
Lx, = test_1d_asym.shape # num points
Lx_mu = AssignQuantity(25, 'microns')
c_r = test_1d_asym.center_reduction/100
sigma = test_1d_asym.sigmastepmax
beta = 0.65 #value from paper 
nu_kin = AssignQuantity(test_1d_asym._extra_vars.get('nu_kin'), 'micron/second') #microns /second
D = AssignQuantity(test_1d_asym._extra_vars.get('D'),'micron^2 / microsecond')

#print('Lx,Ly',Lx,Ly)
print('Lx',Lx)
print('sigma',sigma)
print('beta',beta)
print('nu_kin',nu_kin)
print('D',D)
print('c_r',c_r)

x = figure11(Lx_mu,D,c_r,nu_kin,sigma)
x.ito('microns')
print('x with min values from paper: ',x)
print('predicted number of steps at steady state: ', Nss_fig11(M,B,x.magnitude))


#min values from paper
Lx_mu = AssignQuantity(17.5,'microns')
D = AssignQuantity(0.75E-4,'microns^2 / microsecond')
c_r = 0.15/100
nu_kin = AssignQuantity(20,'microns/second')
sigma = 0.20

#print('Lx,Ly',Lx,Ly)
print('Lx',Lx)
print('sigma',sigma)
print('beta',beta)
print('nu_kin',nu_kin)
print('D',D)
print('c_r',c_r)

x = figure11(Lx_mu,D,c_r,nu_kin,sigma)
x.ito('microns')
print('x with min values from paper: ',x)
print('predicted number of steps at steady state: ', Nss_fig11(M,B,x.magnitude))


#max values from paper
Lx_mu = AssignQuantity(42.5,'microns')
D = AssignQuantity(6E-4,'microns^2 / microsecond')
c_r = 0.7/100
nu_kin = AssignQuantity(140,'microns/second')
sigma = 0.50

#print('Lx,Ly',Lx,Ly)
print('Lx',Lx)
print('sigma',sigma)
print('beta',beta)
print('nu_kin',nu_kin)
print('D',D)
print('c_r',c_r)

x = figure11(Lx_mu,D,c_r,nu_kin,sigma)
x.ito('microns')
print('x with max values from paper: ',x)
print('predicted number of steps at steady state: ', Nss_fig11(M,B,x.magnitude))

In [ ]:
#calculating parameterization produce (fig 11) for simulation
Lx,Ly = test_1d_asym.shape # num points
sigma = test_1d_asym.sigmastepmax
beta = 0.65 #value from paper 
nu_kin = AssignQuantity(test_1d_asym._extra_vars.get('nu_kin'), 'micron/second') #microns /second
D = AssignQuantity(test_1d_asym._extra_vars.get('D'),'micron^2 / microsecond')
c_r = test_1d_asym.center_reduction

#min values from paper
Lx_mu = AssignQuantity(25,'microns')
Ly_mu = Ly/Lx * Lx_mu

x = figure11(Lx_mu,D,c_r,nu_kin,sigma)
x.ito('microns')
print(x)

In [ ]:
#plot surface at middle and end of simulation
plot = test_1d_asym.plot(ice=True,completion=0.2)


In [ ]:
plt.close()

In [ ]:
#continuation of previous simulation
#start a second simulation starting with the ending surface of the first 
# as the initial surface

#create 1d sim and graph number of steps over time
cycle_sim =  Simulation(f2d, (test_1d_asym.shape[0],test_1d_asym.shape[1]), method=test_1d_asym.method, rtol=test_1d_asym.rtol)

#keep general params same
cycle_sim.layermax = 1000 #st_2d_asym.layermax * 1.5 #make sure it reaches steady state
cycle_sim.float_params['DoverdeltaX2'] = test_1d_asym.float_params['DoverdeltaX2'] 
cycle_sim.sigma0 = test_1d_asym.sigma0
cycle_sim.center_reduction = test_1d_asym.center_reduction
cycle_sim.deltaT = test_1d_asym.deltaT #increase time step to decrease number of steps saved?
cycle_sim.noisy_init = test_1d_asym.noisy_init

#set up initial surface
cycle_sim.nonstd_init = True
ntots = test_1d_asym.getNtot()
cycle_sim.startingNtot = ntots[len(ntots)//2] #start with the middle of the last simulation, before discretzation error

#set to ablation run
cycle_sim.sigmastepmax = 0.20 


In [ ]:
#animate ablation of growth ending surface
anim = cycle_sim.animate(ret=True, focus_on_growth=True, proportionalSpeed=False)

In [ ]:
#graph number of steps over time for second simulation
Ntot = cycle_sim.getNtot()
#Ntot = cycle_sim.getNtot()

maxes = np.array([np.max(n) for n in Ntot[:]])
mins = np.array([np.min(n) for n in Ntot[:]])

#print(maxes,mins)
plt.figure()
ax = plt.subplot(111)

ax.set_ylabel('Number of steps on surface')

if cycle_sim.sigmastepmax < 0: #ablation case
    ax.set_xlabel('Total number of layers ablated')
    ax.plot( Ntot[::-1,0,0]+cycle_sim.layermax,maxes-mins) #flipping to show ablation progress from left to right, adding layermax to show # layers ablated instead of negative # layers grown
else: #growth case   
    ax.set_xlabel('Total number of layers grown')
    ax.plot( Ntot[:,0,0],maxes-mins)

plt.grid()